In [1]:
from active_critic.utils.gym_utils import sample_expert_transitions
import torch as th
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
import gym
from imitation.data.wrappers import RolloutInfoWrapper


In [ ]:
def make_env(env):
    def _init():
        env._freeze_rand_vec = False
        #rce = ResetCounterWrapper(env)
        riw = RolloutInfoWrapper(env)
        return riw
    return _init

In [ ]:
def make_vec_env_gym(env, num_cpu, seq_len):
    env = SubprocVecEnv([make_env(env) for i in range(num_cpu)])
    vec_expert = ImitationLearningWrapper(
        policy=policy_dict[env_id][0], env=env)
    return env, vec_expert

In [10]:
env = gym.make('Hopper-v3')

In [1]:
from active_critic.model_src.transformer import *
import torch as th

/home/hendrik/anaconda3/envs/ac/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
seq_len = 6
batch_size = 2
dim = 3

tms = ModelSetup()
tms.d_hid = 12
tms.d_model = 12
tms.d_output = 2
tms.device = 'cuda'
tms.nhead = 1
tms.nlayers = 2
tms.seq_len = seq_len

tm = TransformerModel(model_setup=tms)

In [9]:
inpt = th.ones([batch_size, seq_len, dim], device='cuda')

In [10]:
result = tm.forward(inpt, offset=0)

In [12]:
result.shape

torch.Size([2, 6, 2])